In [22]:
from PIL import Image, ImageDraw

def carregaImagem(sNomeArquivo):
    with open(sNomeArquivo, 'r') as aArquivo:
        aLinhas = aArquivo.readlines()

    sCabecalho        = aLinhas[:4]
    aDimensoes        = sCabecalho[1].split()
    iLargura, iAltura = map(int, aDimensoes)
    iCanal            = int(sCabecalho[2])
    aDados            = []
    for sLinha in aLinhas[3:]:
        aDados.extend(map(int, sLinha.split()))

    return iLargura, iAltura, iCanal, aDados

def salvaImagem(sNomeArquivo, iLargura, iAltura, iCanal, aDados):
    with open(sNomeArquivo, 'w') as oArquivo:
        oArquivo.write(f'P3\n')
        oArquivo.write(f'{iLargura} {iAltura}\n')
        oArquivo.write(f'{iCanal}\n')

        aDadosStr = [str(sValor) for sValor in aDados]
        aLinhas   = [aDadosStr[i:i+iLargura*3] for i in range(0, len(aDadosStr), iLargura*3)]
        for aLinha in aLinhas:
            oArquivo.write(' '.join(aLinha) + '\n')

def redimensionaImagem(oImagem, iNovaLargura, iNovaAltura):
    return oImagem.resize((iNovaLargura, iNovaAltura))

def adicionaImagem(aImg1, aImg2, iX, iY):
    iLargura1, iAltura1, iCanal1, aDados1 = aImg1
    iLargura2, iAltura2, iCanal2, aDados2 = aImg2

    oImg2Redimen = redimensionaImagem(Image.new("RGB", (iLargura1, iAltura1), (0, 0, 0)), iLargura2, iAltura2)

    if iX < 0 or iY < 0 or iX + iLargura2 > iLargura1 or iY + iAltura2 > iAltura1:
        raise ValueError("Coordenadas de início inválidas")

    for i in range(iAltura2):
        for j in range(iLargura2):
            for iCanal in range(3):  # Canais RGB
                iIndice1 = (y + i) * iLargura1 * 3 + (x + j) * 3 + iCanal
                iIndice2 = i * iLargura2 * 3 + j * 3

                aDados1[iIndice1] = min(aDados1[iIndice2] + aDados2[iIndice2 + iCanal], iCanal1)

    return iLargura1, iAltura1, iCanal1, aDados1

oImg1 = carregaImagem("EntradaRgbPai.ppm")
oImg2 = carregaImagem("EntradaRgbAdd.ppm")
iX    = int(input("Informe a coordenada x de início: "))
iY    = int(input("Informe a coordenada y de início: "))

aImagemAdicionada = adicionaImagem(oImg1, oImg2, iX, iY)

salvaImagem("imagemDistintaAdicionada.ppm", *aImagemAdicionada)
